In [6]:
import os
import json
import pandas as pd
import traceback

In [1]:
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'langchain_community'

In [9]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

ModuleNotFoundError: No module named 'dotenv'

In [10]:
KEY=os.getenv("mykey")

In [11]:
print(KEY)

None


In [ ]:
llm=ChatOpenAI(mykey=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [12]:
llm

NameError: name 'llm' is not defined

In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import getn_openai_callback
import PyPDF2

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
#FORMAT IN WHICH WE WANT OUR MCQ
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [ ]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    tmeplate=TEMPLATE
)

In [ ]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt)

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE)

In [ ]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [ ]:
#sequentialchai is a class and genetrate_evaluate_chain is an object
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
file_path=r"C:\Users\PUJA KUMARI\Desktop\mcqgen\data.txt"

In [ ]:
file_path

In [ ]:
#read the data file
with open(file_path,'r') as file:
    TEXT=file.read()

In [ ]:
#inside text all data is present
print(TEXT)

In [ ]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER=5 #no of question we want to genrate
SUBJECT="generative ai"
TONE="simple"

In [ ]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    #calling thus generate_evaluate_chain here
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )


In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz=response.get("quiz")

In [ ]:

quiz=json.loads(quiz)

In [ ]:
#create dataframe in list format
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
#convert to dataframe
#
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
#store this in csv file

quiz.to_csv("generativeai.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')